In [1]:
import requests
import json

In [ ]:
# example call 1
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="DE"][admin_level=2];
(node["amenity"="biergarten"](area);
 way["amenity"="biergarten"](area);
 rel["amenity"="biergarten"](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [ ]:
# example call 2
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="DE"][admin_level=2];
( way["highway"="motorway"](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [3]:
def convert_to_geo_dict(data):
    """TODO"""
    return { "type": "FeatureCollection",
                        "features": [ 
                                        {"type": "Feature",
                                         "geometry": { "type": "LineString",
                                                       "coordinates": [ [point['lon'],
                                                                        point['lat']] for point in feature["geometry"]]},
                                         "properties": { key: value 
                                                         for key, value in feature.items()
                                                         if key != "geometry" }
                                         } 
                                     for feature in data["elements"]
                                    ]
                       }

In [4]:
def query_overpass_api(query):
    """TODO"""
    overpass_url = "http://overpass-api.de/api/interpreter"
    response = requests.get(overpass_url, 
                        params={'data': overpass_query})
    data = response.json()
    return data

In [7]:
overpass_query = """
[out:json];
area[name="Nordrhein-Westfalen"];
way["highway"="motorway"](area);
out geom;
"""
data = query_overpass_api(overpass_query)

In [ ]:
overpass_query = """
[out:json][timeout:25];
area[name="Berlin"];
(
  // query part for: “tourism=attraction”
  node["tourism"="attraction"](area);
  way["tourism"="attraction"](area);
  relation["tourism"="attraction"](area);
);
// print results
out body;
>;
out skel qt;
"""
tourism_attraction_berlin = query_overpass_api(overpass_query)
# TODO: markers for the nodes and not linestrings
geo_json_tourism = convert_to_geo_dict(tourism_attraction_berlin)

In [8]:
# convert overpass api response to geojson
geo_json_data = convert_to_geo_dict(data)

In [9]:
# store 
with open('motorways_nrw_geo.json', 'w') as outfile:
    json.dump(geo_json_data, outfile, indent=4, ensure_ascii=False)